In [2]:
import pyspark

!pip install pyspark findspark


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
    !pip install pyspark[sql]
    import pyspark
    print(pyspark.__version__)


4.0.0



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HW2-Test") \
    .master("local[*]") \
    .getOrCreate()

print("✅ SparkSession created successfully!")
print("Spark version:", spark.version)
print("UI available at:", spark.sparkContext.uiWebUrl)

spark.stop()


✅ SparkSession created successfully!
Spark version: 4.0.0
UI available at: http://10.50.0.152:4040


In [6]:
!pip install findspark
import findspark
findspark.init()



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder \
    .appName("HW2-Test") \
    .master("local[*]") \
    .getOrCreate()

print("SparkSession created")
print("Spark version:", spark.version)
print("UI:", spark.sparkContext.uiWebUrl)

print("\nDone")
#time.sleep(60) # running spark for 1 minute

#spark.stop()

SparkSession created
Spark version: 4.0.0
UI: http://10.50.0.152:4040

Done


Dataset Check

In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SalesRecordsAnalysis") \
    .master("local[*]") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

data_path = r"C:\Users\ASUS\PycharmProjects\Python\Big_Data\100000 Sales Records.csv"
df_raw = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .csv(data_path))

print("Rows:", df_raw.count())
df_raw.printSchema()
df_raw.show(5, truncate=False)

Rows: 100000
root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)

+---------------------------------+---------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Region                           |Country              |Item Type    |Sales Channel|Order Priority|Order Date|Order ID |Ship Date |Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost

RDD Part

In [20]:
from pyspark.sql import functions as F

profit_check = (df_raw
    .select(
        F.sum(F.when(F.col("Total Profit") > 0, 1).otherwise(0)).alias("positive_profits"),
        F.sum(F.when(F.col("Total Profit") == 0, 1).otherwise(0)).alias("zero_profits"),
        F.sum(F.when(F.col("Total Profit") < 0, 1).otherwise(0)).alias("negative_profits")
    )
)

profit_check.show()

+----------------+------------+----------------+
|positive_profits|zero_profits|negative_profits|
+----------------+------------+----------------+
|          100000|           0|               0|
+----------------+------------+----------------+



In [25]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SalesRecordsAnalysis") \
    .master("local[*]") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.python.worker.reuse", "false") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executorEnv.PYTHONUTF8", "1") \
    .getOrCreate()


In [27]:
df_raw = spark.read.option("header", True).option("inferSchema", True).csv(r"C:\Users\ASUS\PycharmProjects\Python\Big_Data\100000 Sales Records.csv")


In [28]:
rdd = df_raw.rdd.map(lambda row: row.asDict())
print(rdd.take(3))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 24) (10.50.0.152 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:945)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRun

In [29]:
import os

# ---- Force Spark to use direct python daemon instead of new worker processes ----
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--conf spark.python.profile=true pyspark-shell"

# ---- Optional: increase memory and disable Arrow optimizations ----
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("SalesRecordsAnalysis")
    .master("local[*]")
    .config("spark.sql.execution.arrow.pyspark.enabled", "false")
    .config("spark.python.worker.reuse", "false")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "2g")
    .getOrCreate()
)

print("✅ SparkSession created successfully!")
print("Spark version:", spark.version)
print("UI available at:", spark.sparkContext.uiWebUrl)

✅ SparkSession created successfully!
Spark version: 4.0.0
UI available at: http://10.50.0.152:4040


In [30]:
path = r"C:\Users\ASUS\PycharmProjects\Python\Big_Data\100000 Sales Records.csv"

df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(path)
)

print("✅ Rows:", df_raw.count())
df_raw.printSchema()
df_raw.show(5, truncate=False)


✅ Rows: 100000
root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)

+---------------------------------+---------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Region                           |Country              |Item Type    |Sales Channel|Order Priority|Order Date|Order ID |Ship Date |Units Sold|Unit Price|Unit Cost|Total Revenue|Total Co

In [32]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("CheckSpark").getOrCreate()

df = spark.read.csv("100000 Sales Records.csv", header=True, inferSchema=True)
df.show(3)
print(df.rdd.take(3))

spark.stop()


+--------------------+--------------------+---------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Middle East and N...|          Azerbaijan|   Snacks|       Online|             C| 10/8/2014|535113847|10/23/2014|       934|    152.58|    97.44|    142509.72|  91008.96|    51500.76|
|Central America a...|              Panama|Cosmetics|      Offline|             L| 2/22/2015|874708545| 2/27/2015|      4551|     437.2|   263.33|    1989697.2|1198414.83|   791282.37|
|  Sub-Saharan Africa|Sao Tome and Prin...|   Fruits|      Offline|        

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 40) (10.50.0.152 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:945)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRun

In [23]:
# first I have to convert DataFrame to RDD of Rows for the clarity
rdd = df_raw.select("Region", "Item Type", "Units Sold", "Total Profit").rdd

# first I am using map() transformation to derive Region, Item Type and Profit
def safe_float(x):
    try:
        return float(x) if x is not None else 0.0
    except:
        return 0.0

mapped = rdd.map(lambda r: (r["Region"], (r["Item Type"], safe_float(r["Total Profit"]))))

# filter() to keep only valid rows (non-null profit and units sold > 0)
filtered = mapped.filter(lambda t: t[1][1] > 0)

# next, flatMap(), here I split into (Region, Profit) and (Region, ItemType) pairs for flexibility and demonstrate multiple outputs per record
flat_mapped = filtered.flatMap(lambda t: [(t[0], t[1][1]), (t[0], 1)])

# then reduceByKey() to aggregate total profit by region
region_profit = filtered.map(lambda t: (t[0], t[1][1])) \
                        .reduceByKey(lambda a, b: a + b)

print(region_profit.take(10))

# take()
sample = region_profit.take(10)
print("Sample region–profit pairs:", sample)

# In order to analyze the data properly I have also ranked top-5 regions by total profit
top_regions = region_profit.takeOrdered(5, key=lambda x: -x[1])
print("\nTop 5 profitable regions:")
for region, profit in top_regions:
    print(f"{region:35s} {profit:,.2f}")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 11.0 failed 1 times, most recent failure: Lost task 1.0 in stage 11.0 (TID 17) (10.50.0.152 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:945)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRun

In [ ]:
df_raw = df

from pyspark.sql import functions as F

# first I have to convert DataFrame to RDD of Rows for the clarity
rdd = df_raw.select("Region", "Item Type", "Units Sold", "Total Profit").rdd

# first I am using map() transformation to derive Region, Item Type and Profit
def safe_float(x):
    try:
        return float(x) if x is not None else 0.0
    except:
        return 0.0

mapped = rdd.map(lambda r: (r["Region"], (r["Item Type"], safe_float(r["Total Profit"]))))

# filter() to keep only valid rows (non-null profit and units sold > 0)
filtered = mapped.filter(lambda t: t[1][1] > 0)

# next, flatMap(), here I split into (Region, Profit) and (Region, ItemType) pairs for flexibility and demonstrate multiple outputs per record
flat_mapped = filtered.flatMap(lambda t: [(t[0], t[1][1]), (t[0], 1)])

# then reduceByKey() to aggregate total profit by region
region_profit = filtered.map(lambda t: (t[0], t[1][1])) \
                        .reduceByKey(lambda a, b: a + b)

# take()
sample = region_profit.take(10)
print("Sample region–profit pairs:", sample)

# In order to analyze the data properly I have also ranked top-5 regions by total profit
top_regions = region_profit.takeOrdered(5, key=lambda x: -x[1])
print("\nTop 5 profitable regions:")
for region, profit in top_regions:
    print(f"{region:35s} {profit:,.2f}")

In [ ]:
df_dates = (df
    .withColumn("OrderDate", F.to_date(F.col("Order Date"), "M/d/yyyy"))
    .withColumn("Year", F.year("OrderDate"))
    .withColumn("Month", F.month("OrderDate"))
)

monthly_profit = (df_dates
    .groupBy("Year", "Month")
    .agg(F.sum("Total Profit").alias("Total_Profit"))
    .orderBy("Year", "Month")
)
monthly_profit.show(12, truncate=False)

In [ ]:
from pyspark.sql import functions as F

text_cols = ["Region","Country","Item Type","Sales Channel","Order Priority","Order Date","Ship Date"]

df1 = df.select(
    *[F.trim(F.col(c)).alias(c) if c in text_cols else F.col(c) for c in df.columns]
).withColumn("Region", F.initcap("Region")) \
 .withColumn("Country", F.initcap("Country")) \
 .withColumn("Item Type", F.initcap("Item Type")) \
 .withColumn("Sales Channel", F.initcap("Sales Channel")) \
 .withColumn("Order Priority", F.upper("Order Priority"))

df1.select("Region","Country","Item Type","Sales Channel","Order Priority").show(5, truncate=False)


In [ ]:
top_lines = (df_clean
    .select("Region","Country","Item Type","Units Sold","Total Profit")
    .orderBy(F.desc("Total Profit"), F.desc("Units Sold"))
)
top_lines.show(10, truncate=False)